In [20]:
from fastapi import FastAPI, Request, HTTPException, BackgroundTasks
from fastapi.responses import JSONResponse
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import numpy as np
import io
import base64
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import time
from fastapi.middleware.cors import CORSMiddleware


In [22]:
# FastAPI 應用
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 你可以根據需求設置具體允許的域名
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
# 設定 Google Sheets API 的範圍
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# 加載服務帳戶憑證
creds = Credentials.from_service_account_file("credentials.json", scopes=SCOPES)
client = gspread.authorize(creds)

# 打開你的 Google Sheets
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1n3Xt5iemJ3WWjlksEaU6JfNDv_9FX4CStTzjayMRwEU/edit#gid=0")

# 定義資料處理函數
def calculate_avg(df):
    avg_dict = {}
    unique_positions = sorted(df['position'].unique())
    unique_names = df['name'].unique()

    for position in unique_positions:
        avg_dict[position] = {}
        for name in unique_names:
            single_count = (df[(df['name'] == name) & (df['position'] == position)]['result'] == '一安 Single').sum()
            double_count = (df[(df['name'] == name) & (df['position'] == position)]['result'] == '二安 Double').sum()
            triple_count = (df[(df['name'] == name) & (df['position'] == position)]['result'] == '三安 Triple').sum()
            hr_count = (df[(df['name'] == name) & (df['position'] == position)]['result'] == '全壘打 Home Run').sum()
            total_hits = single_count + double_count + triple_count + hr_count
            total_at_bats = (df[(df['name'] == name) & (df['position'] == position)]['result']
                             .isin(['一安 Single', '二安 Double', '三安 Triple', '全壘打 Home Run', '外飛 Field Out', '失誤 Field Error', '三振 Strikeout']) 
                             & ~df[(df['name'] == name) & (df['position'] == position)]['result'].isna()).sum()

            if total_at_bats > 0:
                avg = total_hits / total_at_bats
                avg_dict[position][name] = avg
            else:
                avg_dict[position][name] = None

    avg_df = pd.DataFrame(avg_dict)
    avg_df.index.name = 'name'
    return avg_df

In [24]:
@app.get("/")
async def get_data(n: str, width: int, background_tasks: BackgroundTasks):
    try:
        worksheet = spreadsheet.worksheet(n)
        values = worksheet.get_all_values()
        truncated_values = [row[:width] for row in values]

        # # 在讀取資料時觸發更新
        # background_tasks.add_task(refresh_data)

        return JSONResponse(content=truncated_values)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
response = requests.get("http://localhost:8000/?n=image&width=2")
print(response.status_code)
print(response.json())